# Desafio 1: utilizando parâmetros do json_normalize

Ao normalizar o objeto JSON durante a aula passamos simplesmente `pd.json_normalize(<dados>)` mas o método json_normalize possui diversos parâmetros para trabalhar com estruturas de dados mais complexas.

Você recebeu a tarefa de normalizar o seguinte arquivo JSON chamado “informacoes.json” referente a algumas informações de identificação de um cliente. 

In [1]:
import pandas as pd
from json import load


In [7]:
with open('in/informacoes.json') as f:
    json = load(f)

pd.json_normalize(json, record_path='enderecos', meta=['nome', 'idade'])


,tipo,rua,numero,cidade,nome,idade
0,casa,Rua A,123,São Paulo,João,28
1,trabalho,Rua B,456,Rio de Janeiro,João,28


# Desafio 2: tratando os dados

Sua tarefa é realizar alguns tratamentos e deixar esse arquivo JSON como um Pandas DataFrame, sem dados vazios e com a tipagem das colunas de forma correta. Como isso pode ser feito?



In [16]:
with open('in/dados.json') as f:
    json = load(f)

dados = pd.json_normalize(json, record_path='pessoas').explode('telefones').reset_index(drop=True)
dados


,nome,idade,telefones,endereco.rua,endereco.numero,endereco.cidade
0,João,25,11 1111-1111,Rua A,123,São Paulo
1,João,25,11 2222-2222,Rua A,123,São Paulo
2,Maria,30,21 3333-3333,,456,Rio de Janeiro


In [20]:
dados['idade'] = dados['idade'].astype(int)
dados.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nome             3 non-null      object
 1   idade            3 non-null      int64 
 2   telefones        3 non-null      object
 3   endereco.rua     3 non-null      object
 4   endereco.numero  3 non-null      int64 
 5   endereco.cidade  3 non-null      object
dtypes: int64(2), object(4)
memory usage: 276.0+ bytes


In [25]:
subset_dados = dados.query('`endereco.rua` != ""').reset_index(drop=True).copy()
subset_dados


,nome,idade,telefones,endereco.rua,endereco.numero,endereco.cidade
0,João,25,11 1111-1111,Rua A,123,São Paulo
1,João,25,11 2222-2222,Rua A,123,São Paulo


# Desafio 3: tratando uma base de dados com dados duplicados e nulos


Para resolver isso, foi solicitado que você identifique e remova as amostras que apresentam os seguintes problemas:

- Valores nulos: são valores ausentes em algumas das colunas que devem ser preenchidos para que a análise seja correta.
- Duplicatas: registros iguais que podem prejudicar a análise dos dados, já que estão representando a mesma informação.
- Strings vazias: valores de texto que não apresentam nenhum conteúdo escrito, o que pode dificultar a análise de dados, principalmente se a coluna tiver muitos valores assim.
- Conversão de tipos: algumas colunas do arquivo podem estar no tipo de dados errado, como texto em vez de número, o que pode prejudicar a análise. Nesse caso você deve realizar a conversão para o tipo correto de cada coluna.
Dentro desse contexto, você precisará de uma abordagem sistemática para limpar os dados. Assim, como isso pode ser feito?



In [32]:
with open('in/cursos_cadastrados.json') as f:
    json = load(f)

cursos_cadastrados = pd.json_normalize(json)

cursos_cadastrados.head()


,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.90,João Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Excel para iniciantes,Produtividade,None,None,None,None,None,None,Maria Oliveira,maria.oliveira@emailaleatorio.com,(11) 8888-8888
2,Marketing digital para negócios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutório em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777
3,Inteligência artificial,Programação,40 horas,,2022-04-01,,Curso avançado sobre inteligência artificial c...,129.9,,contato@emailaleatorio.com,
4,Inglês para negócios,Idiomas,20 horas,30,,,Curso de inglês para negócios,69.9,John Smith,,


In [35]:
cursos_cadastrados[cursos_cadastrados.isna().any(axis=1)]


,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
1,Excel para iniciantes,Produtividade,None,None,None,None,None,None,Maria Oliveira,maria.oliveira@emailaleatorio.com,(11) 8888-8888


In [38]:
subset_sem_nulos = cursos_cadastrados.dropna().reset_index(drop=True).copy()
subset_sem_nulos


,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.90,João Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negócios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutório em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777
2,Inteligência artificial,Programação,40 horas,,2022-04-01,,Curso avançado sobre inteligência artificial c...,129.9,,contato@emailaleatorio.com,
3,Inglês para negócios,Idiomas,20 horas,30,,,Curso de inglês para negócios,69.9,John Smith,,
4,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.90,João Silva,joao.silva@emailaleatorio.com,(11) 9999-9999


In [41]:
subset_sem_nulos[subset_sem_nulos.duplicated()]


,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
4,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.90,João Silva,joao.silva@emailaleatorio.com,(11) 9999-9999


In [44]:
subset_sem_duplicatas = subset_sem_nulos.drop_duplicates().reset_index(drop=True).copy()
subset_sem_duplicatas


,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.90,João Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negócios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutório em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777
2,Inteligência artificial,Programação,40 horas,,2022-04-01,,Curso avançado sobre inteligência artificial c...,129.9,,contato@emailaleatorio.com,
3,Inglês para negócios,Idiomas,20 horas,30,,,Curso de inglês para negócios,69.9,John Smith,,


In [54]:
cols = subset_sem_duplicatas.columns.to_list()
subset_sem_vazio = subset_sem_duplicatas.copy()

for col in cols:
    if '' in subset_sem_duplicatas[col].unique():
        subset_sem_vazio = subset_sem_vazio[subset_sem_vazio[col] != '']

subset_sem_vazio


,curso,categoria,carga_horaria,concluintes,data_inicio,data_conclusao,descricao,preco,instrutor.nome,instrutor.email,instrutor.telefone
0,Introdução à programação,Programação,20 horas,100,2022-01-01,2022-01-20,Curso introdutório à programação com Python,99.90,João Silva,joao.silva@emailaleatorio.com,(11) 9999-9999
1,Marketing digital para negócios,Marketing,30 horas,75,2022-03-01,2022-03-31,Curso introdutório em marketing,89.9,Ana Santos,ana.santos@emailaleatorio.com,(11) 7777-7777


In [55]:
subset_sem_vazio.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   curso               2 non-null      object
 1   categoria           2 non-null      object
 2   carga_horaria       2 non-null      object
 3   concluintes         2 non-null      object
 4   data_inicio         2 non-null      object
 5   data_conclusao      2 non-null      object
 6   descricao           2 non-null      object
 7   preco               2 non-null      object
 8   instrutor.nome      2 non-null      object
 9   instrutor.email     2 non-null      object
 10  instrutor.telefone  2 non-null      object
dtypes: object(11)
memory usage: 192.0+ bytes


In [59]:
subset_sem_vazio['concluintes'] = subset_sem_vazio['concluintes'].astype(int)
subset_sem_vazio['data_inicio'] = pd.to_datetime(subset_sem_vazio['data_inicio'])
subset_sem_vazio['data_conclusao'] = pd.to_datetime(subset_sem_vazio['data_conclusao'])
subset_sem_vazio['preco'] = subset_sem_vazio['preco'].astype(float)

subset_sem_vazio.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2 entries, 0 to 1
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   curso               2 non-null      object        
 1   categoria           2 non-null      object        
 2   carga_horaria       2 non-null      object        
 3   concluintes         2 non-null      int64         
 4   data_inicio         2 non-null      datetime64[ns]
 5   data_conclusao      2 non-null      datetime64[ns]
 6   descricao           2 non-null      object        
 7   preco               2 non-null      float64       
 8   instrutor.nome      2 non-null      object        
 9   instrutor.email     2 non-null      object        
 10  instrutor.telefone  2 non-null      object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 192.0+ bytes


# Desafio 4: Lidando com Outliers

Foi informado que nessa base de dados há possíveis fraudes que podem ser identificadas através dos outliers no “valor da transação”. Basicamente a fraude seria um outlier.

Sua missão é construir uma função que recebe dois parâmetros, a coluna que você quer analisar se tem outlier e o DataFrame em questão. A intenção de fazer isso é aproveitar novamente essa função tanto para outras colunas quanto para outros DataFrames futuramente em outras análises. O método que deve ser implementado para identificação do outlier é do intervalo interquartil.

Como podemos fazer isso?

In [1]:
import pandas as pd

# criando um DataFrame com 30 transações aleatórias
df = pd.DataFrame({
    'ID da transação': range(1, 31),
    'Valor da transação': [100, 200, 150, 500, 300, 913, 250, 400, 200, 150, 
                           200, 200, 400, 300, 150, 301, 805, 300, 400, 250, 
                           150, 100, 500, 600, 200, 350, 100, 250, 800, 250],
    'Data da transação': pd.date_range(start='2022-01-01', end='2022-01-30', freq='D'),
    'Local da transação': ['São Paulo, Brasil', 'Rio de Janeiro, Brasil', 'Belo Horizonte, Brasil', 'São Paulo, Brasil', 
                           'São Paulo, Brasil', 'Nova Iorque, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'Rio de Janeiro, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Los Angeles, EUA', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil', 'São Paulo, Brasil',
                           'São Paulo, Brasil', 'São Paulo, Brasil', 'Miami, EUA', 'São Paulo, Brasil']
})


In [18]:
df = df.set_index('ID da transação')


In [19]:
def outliers_filter(column: pd.Series):
    q1 = column.quantile(0.25)
    q3 = column.quantile(0.75)
    iqr = q3 - q1

    li = q1 - iqr * 1.5
    ls = q3 + iqr * 1.5

    return (column < li) | (column > ls)

filtro = outliers_filter(df['Valor da transação'])
df[filtro]


,Valor da transação,Data da transação,Local da transação
ID da transação,,,
6,913,2022-01-06,"Nova Iorque, EUA"
17,805,2022-01-17,"Los Angeles, EUA"
29,800,2022-01-29,"Miami, EUA"
